In [1]:
from azureml.core import Workspace, Experiment, Environment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")
env = Environment.get(workspace=ws, name='AzureML-Minimal')
 
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FZREEL7KT to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-141132
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-141132


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# cpu_cluster_name is just that: a name. NOthing more, nothing less.
cpu_cluster_name= "udacity-project"
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# additional code, that is not needed for our cluster to work, but gives us info about 
# the status and properties of the new cluster

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-22T13:36:38.929000+00:00', 'errors': None, 'creationTime': '2021-03-22T13:36:35.975275+00:00', 'modifiedTime': '2021-03-22T13:36:51.412208+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:

# Specify parameter sampler
# we are looking for a discrete variable, "yes" or "no"
# choose random sweep for efficiency; faster than grid search and equally effective. 
# Bayesian is out of the question, since it does not support an early termination policy



# other possibility; need to check


# Create a SKLearn estimator for use with train.py
from azureml.core import ScriptRunConfig
# configure and submit your training run
config = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=compute_target,
                            environment=None)
script_run = exp.submit(config)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=config,
                             hyperparameter_sampling=ps,
                             policy= policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_duration_minutes = 100,
                             max_total_runs=100,
                             max_concurrent_runs=4)


In [5]:
# test
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

experiment = Experiment(ws, 'quick-starts-ws-141114')

ps = RandomParameterSampling({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200)
})
# Specify an early termination Policy. Early termination improves computational efficiency.
# Bandit is an aggresive policy, saving a lot of time but mabe missing some promising candidates

policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=1, delay_evaluation=5)



ps = RandomParameterSampling({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200)
})
# Specify an early termination Policy. Early termination improves computational efficiency.
# Bandit is an aggresive policy, saving a lot of time but mabe missing some promising candidates

policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./traning")

script_folder = "./training"
os.makedirs(script_folder, exist_ok=True)
shutil.copy("./train.py", script_folder)

est = SKLearn(source_directory = script_folder, entry_script="train.py",compute_target=compute_target, vm_size="Standard_d2_V2", vm_priority="lowpriority")

hyperdrive_config = HyperDriveConfig(hyperparameter_sampling= ps,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    policy = policy,
                                    estimator=est)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment

hyperdrive_run = exp.submit(config=hyperdrive_config)

# visualize with widget
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [26]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()


print('Best Run Id: ', best_run.id)
print('\n Best run metrics:', best_run_metrics)
print('\n Filename:', best_run.get_file_names())
print('\n Run details:',parameter_values)

model = best_run.register_model(model_name="hyerdrive_log_reg_best_run",
model_path="outputs/model.joblib",
tags={"Method":"Hyperdrive"},
properties={"Accuracy": best_run.get_metrics()["Accuracy"]})

Best Run Id:  HD_417e6bef-6d5e-488f-bdfe-1fe8029ad5cb_3

 Best run metrics: {'Regularization Strength:': 0.1, 'Max iterations:': 20, 'Accuracy': 0.908649468892261}

 Filename: ['azureml-logs/55_azureml-execution-tvmps_a71995b1f87704279f2d1c0aa3fc768831f1d40cbfe9f232b5127e899b8403de_d.txt', 'azureml-logs/65_job_prep-tvmps_a71995b1f87704279f2d1c0aa3fc768831f1d40cbfe9f232b5127e899b8403de_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_a71995b1f87704279f2d1c0aa3fc768831f1d40cbfe9f232b5127e899b8403de_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']

 Run details: {'runId': 'HD_417e6bef-6d5e-488f-bdfe-1fe8029ad5cb_3', 'target': 'udacity-project', 'status': 'Completed', 'startTimeUtc': '2021-03-22T13:47:58.99216Z', 'endTimeUtc': '2021-03-22T13:48:51.143959Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapsh

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_a71995b1f87704279f2d1c0aa3fc768831f1d40cbfe9f232b5127e899b8403de_d.txt', 'azureml-logs/65_job_prep-tvmps_a71995b1f87704279f2d1c0aa3fc768831f1d40cbfe9f232b5127e899b8403de_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_a71995b1f87704279f2d1c0aa3fc768831f1d40cbfe9f232b5127e899b8403de_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/model.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_a71995b1f

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=66)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='spearman_correlation',
    training_data=X_train,
    label_column_name="y",
    n_cross_validations=)

In [2]:
# Submit your automl run

experiment = Experiment(ws, "automl_run")
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)